# DeepLabCut example for single animal project - BacciLab

In [1]:
import sys
sys.path.append("../..")
import deeplabcut

import plotly.express as px
from pathlib import Path
import pandas as pd
import os

%gui qt

Loading DLC 3.0.0rc13...


In [ ]:
video_path = r"" # for creating a new project
project_location = r"" # for creating a new project

config_path = r"" # to use an existing project

### Create a project

In [ ]:
config_path = deeplabcut.create_new_project(
    project='myproject',
    experimenter='experiementer',
    videos=list(Path(video_path).glob('*.mp4')),
    working_directory=Path(project_location),
    multianimal=False,
    copy_videos=False,
)

config_path = Path(config_path)

### Extracting Frames

For automatic frame extraction, use cell first below. Thre is two algorithm available uniform (faster) and kmeans (clustering) which would be recommended only if videos are short. It is recommended to extract frames automatically the first time for convenience. It will be always possible to complete training set by extracting frames manually (second cell).

In [ ]:
# Extracting frames automatically
deeplabcut.extract_frames(
    config_path,
    mode='automatic',
    algo='uniform',
    userfeedback=False,
)

In [ ]:
# Extracting frames manually
video_path = r"video.mp4"
deeplabcut.extract_frames(
    config_path, 
    "manual", 
    videos_list=[video_path]
)

### Labelling Frames

In [ ]:
# Labelling frames of a specific folder
image_folder = r"myproject\labeled-data\video"
_ = deeplabcut.label_frames(config_path, image_folder=image_folder)

In [ ]:
# Labelling frames of all folders in a loop
labeled_data_dir = os.path.join(os.path.dirname(config_path), 'labeled-data')
labeled_folders = [foldername for foldername in os.listdir(labeled_data_dir) if '_labeled' not in foldername]

deeplabcut.label_frames_in_loop(config_path, labeled_folders)

In [ ]:
deeplabcut.SkeletonBuilder(config_path)

In [ ]:
deeplabcut.check_labels(config_path)

### Create the training dataset

Choose a proper backbone for the job.

In [2]:
deeplabcut.create_training_dataset?

Signature:
deeplabcut.create_training_dataset(
    config,
    num_shuffles=1,
    Shuffles=None,
    windows2linux=False,
    userfeedback=True,
    trainIndices=None,
    testIndices=None,
    net_type=None,
    detector_type=None,
    augmenter_type=None,
    posecfg_template=None,
    superanimal_name='',
    weight_init: 'WeightInitialization | None' = None,
    engine: 'Engine | None' = None,
    ctd_conditions: 'int | str | Path | tuple[int, str] | tuple[int, int] | None' = None,
)
Docstring:
Creates a training dataset.

Labels from all the extracted frames are merged into a single .h5 file.
Only the videos included in the config file are used to create this dataset.

Parameters
----------
config : string
    Full path of the ``config.yaml`` file as a string.

num_shuffles : int, optional, default=1
    Number of shuffles of training dataset to create, i.e. ``[1,2,3]`` for
    ``num_shuffles=3``.

Shuffles: list[int], optional
    Alternatively the user can also give a list of s

In [ ]:
deeplabcut.create_training_dataset(
    config_path,
    net_type='top_down_hrnet_w32'
)

Go to the pytorch_config.yaml to tune training parameters.

### Training the network

In [3]:
deeplabcut.train_network?

Signature:
deeplabcut.train_network(
    config: 'str | Path',
    shuffle: 'int' = 1,
    trainingsetindex: 'int' = 0,
    max_snapshots_to_keep: 'int | None' = None,
    displayiters: 'int | None' = None,
    saveiters: 'int | None' = None,
    maxiters: 'int | None' = None,
    epochs: 'int | None' = None,
    save_epochs: 'int | None' = None,
    allow_growth: 'bool' = True,
    gputouse: 'str | None' = None,
    autotune: 'bool' = False,
    keepdeconvweights: 'bool' = True,
    modelprefix: 'str' = '',
    superanimal_name: 'str' = '',
    superanimal_transfer_learning: 'bool' = False,
    engine: 'Engine | None' = None,
    device: 'str | None' = None,
    snapshot_path: 'str | Path | None' = None,
    detector_path: 'str | Path | None' = None,
    batch_size: 'int | None' = None,
    detector_batch_size: 'int | None' = None,
    detector_epochs: 'int | None' = None,
    detector_save_epochs: 'int | None' = None,
    pose_threshold: 'float | None' = 0.1,
    pytorch_cfg_updates:

In [ ]:
deeplabcut.train_network(
    config_path,
    shuffle=5,
)

### Evaluate the network

In [4]:
deeplabcut.evaluate_network?

Signature:
deeplabcut.evaluate_network(
    config: 'str | Path',
    Shuffles: 'Iterable[int]' = (1,),
    trainingsetindex: 'int | str' = 0,
    plotting: 'bool | str' = False,
    show_errors: 'bool' = True,
    comparisonbodyparts: 'str | list[str]' = 'all',
    gputouse: 'str | None' = None,
    rescale: 'bool' = False,
    modelprefix: 'str' = '',
    per_keypoint_evaluation: 'bool' = False,
    snapshots_to_evaluate: 'list[str] | None' = None,
    pcutoff: 'float | list[float] | dict[str, float] | None' = None,
    engine: 'Engine | None' = None,
    **torch_kwargs,
)
Docstring:
Evaluates the network.

Evaluates the network based on the saved models at different stages of the training
network. The evaluation results are stored in the .h5 and .csv file under the
subdirectory 'evaluation_results'. Change the snapshotindex parameter in the config
file to 'all' in order to evaluate all the saved models.

Parameters
----------
config : string
    Full path of the config.yaml file.

S

In [ ]:
deeplabcut.evaluate_network(
    config_path,
    Shuffles=[5],
    plotting=False,
    per_keypoint_evaluation=True,
    snapshots_to_evaluate=['snapshot-175']
)

### Analyze the videos

In [5]:
deeplabcut.analyze_videos?

Signature:
deeplabcut.analyze_videos(
    config: 'str',
    videos: 'list[str]',
    videotype: 'str' = '',
    shuffle: 'int' = 1,
    trainingsetindex: 'int' = 0,
    gputouse: 'str | None' = None,
    save_as_csv: 'bool' = False,
    in_random_order: 'bool' = True,
    destfolder: 'str | None' = None,
    batchsize: 'int' = None,
    cropping: 'list[int] | None' = None,
    TFGPUinference: 'bool' = True,
    dynamic: 'tuple[bool, float, int]' = (False, 0.5, 10),
    modelprefix: 'str' = '',
    robust_nframes: 'bool' = False,
    allow_growth: 'bool' = False,
    use_shelve: 'bool' = False,
    auto_track: 'bool' = True,
    n_tracks: 'int | None' = None,
    animal_names: 'list[str] | None' = None,
    calibrate: 'bool' = False,
    identity_only: 'bool' = False,
    use_openvino: 'str | None' = None,
    engine: 'Engine | None' = None,
    **torch_kwargs,
)
Docstring:
Makes prediction based on a trained network.

The index of the trained network is specified by parameters in the 

In [ ]:
videos_to_analyze = Path(r"video.mp4")
deeplabcut.analyze_videos(
    config_path,
    videos=[videos_to_analyze],
    shuffle=5
)

In [ ]:
deeplabcut.filterpredictions(
    config_path,
    video=[str(videos_to_analyze)],
    shuffle=5,
)

#### Plot trajectories

In [ ]:
hdf_filepath = r"myproject/videos/video.h5"
df = pd.read_hdf(hdf_filepath)
df = df.droplevel('scorer', axis=1)

fs = 30  # video framerate
df.index = pd.MultiIndex.from_arrays([df.index, df.index / fs], names=['frame','time']) # create a MultiIndex for rows with frame and time

In [ ]:
plot_df = df.melt(ignore_index=False).query('coords != "likelihood"').reset_index()

fig = px.line(plot_df, x='time', y='value', color='bodyparts', line_dash='coords',
              title="Bodyparts' coordinates over time", labels={'value':'coordinate (pixels)', 'time':'time (s)'})
fig.update_traces(marker=dict(size=2))
fig.show()

In [ ]:
plot_df = df.stack(['bodyparts'], future_stack=True).reset_index().drop(['frame', 'time'], axis=1)

fig = px.scatter(plot_df, x='x', y='y', color='bodyparts', height=800,
                 title="Positions of bodyparts in space")
fig.update_traces(marker=dict(size=2))
fig.update_xaxes(scaleanchor="y", scaleratio=1)
fig.update_yaxes(autorange="reversed")
fig.show()

In [ ]:
plot_df = df.melt(ignore_index=False).query('coords == "likelihood"').reset_index()

fig = px.line(plot_df, x='time', y='value', color='bodyparts', line_dash='coords',
              title="Bodyparts' likelihood over time", labels={'value':'likelihood', 'time':'time (s)'})
fig.update_traces(marker=dict(size=2))
fig.show()

#### Create video for visualization

In [ ]:
deeplabcut.create_labeled_video(
    config_path,
    videos=[str(videos_to_analyze)],
    draw_skeleton=True,
    color_by='bodypart',
    shuffle=5,
    filtered=True,
)